In [1]:
import os
from pathlib import Path

# Document folder

In [2]:
doc_dir = "doc_folder_1"

files = list(Path(doc_dir).glob("*")) 
print(files)

[PosixPath('doc_folder_1/2408.09869v5 copy.pdf'), PosixPath('doc_folder_1/.DS_Store')]


In [3]:
file = files[0]
print(file)

doc_folder_1/2408.09869v5 copy.pdf


# Basic usage

Using `DocumentConverter` to process a document.

In [4]:
from docling.document_converter import DocumentConverter

In [5]:
source = file
converter = DocumentConverter()
doc = converter.convert(source).document

/Users/pisek/miniconda3/envs/rag-multimodal/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Serialize the converted result with `export_to_...` command

In [6]:
print(doc.export_to_markdown()[:500]) 

<!-- image -->

## Docling Technical Report

Version 1.0

Christoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis Matteo Omenetti Fabian Lindlbauer Kasper Dinkla Lokesh Mishra Yusik Kim Shubham Gupta Rafael Teixeira de Lima Valery Weber Lucas Morin Ingmar Meijer Viktor Kuropiatnyk Peter W. J. Staar

AI4K Group, IBM Research R¨ uschlikon, Switzerland

## Abstract

This technical report introduces Docling , an easy to use, self-contained, MITli


# Annotate images with OpenAI API call

In [7]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    PictureDescriptionApiOptions
)
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling_core.types.doc.document import ImageRefMode

def convert_with_image_annotation(input_doc_path):
    api_key = os.environ.get("OPENAI_API_KEY")
    model = "gpt-4.1-mini"
    picture_desc_api_option = PictureDescriptionApiOptions(
        url="https://api.openai.com/v1/chat/completions",
        prompt="Describe this image in sentences in a single paragraph.",
        params=dict(
                model=model,
            ),
        headers={
            "Authorization": "Bearer " + api_key,
        },
        timeout=60,
    )

    pipeline_options = PdfPipelineOptions(
        do_picture_description=True,
        picture_description_options=picture_desc_api_option,
        enable_remote_services=True,
        generate_picture_images=True,
        images_scale=2,
    )

    converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
    )
    conv_res = converter.convert(source=input_doc_path)
    return conv_res

In [8]:
result = convert_with_image_annotation(file)

In [9]:
print(result.document.export_to_markdown(mark_annotations = True, include_annotations=True))

<!-- image -->

## Docling Technical Report

Version 1.0

Christoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis Matteo Omenetti Fabian Lindlbauer Kasper Dinkla Lokesh Mishra Yusik Kim Shubham Gupta Rafael Teixeira de Lima Valery Weber Lucas Morin Ingmar Meijer Viktor Kuropiatnyk Peter W. J. Staar

AI4K Group, IBM Research R¨ uschlikon, Switzerland

## Abstract

This technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. The code interface allows for easy extensibility and addition of new features and models.

## 1 Introduction

Converting PDF documents back into a machine-processable format has been a major challenge for decades due to their huge variabi

# Save to MD with external referenced images

In [10]:
def export_function_md_with_image_ref(conv_res, output_path:str, replace_blank:str="_"):
    
    output_dir = Path(output_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem.replace(" ", replace_blank)

    # Save markdown with externally referenced pictures
    md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED, include_annotations=True)

In [11]:
export_function_md_with_image_ref(result, "outputs")